In [ ]:
import numpy as np  
import pandas 
 
import uproot as ur    

#for reading in the dataset, only having the phi track beacsue of storage space. 
 
file = ur.open(r"C:\Users\keega\Downloads\com_8160_minpt_0_merged_tracks (1).root")

# List all keys (e.g., trees, histograms)
print(file.keys())  

# Access a TTree 
tree = file["jet_tree"]['vtrackphi']  # Replace with actual tree name

# Convert to a pandas DataFrame
df = tree.arrays(library="pd")  
del tree

vtrackphi_test=df.to_numpy()  
del df

weights = file["jet_tree"]['weight'] 
weightss = weights.arrays(library="pd")
weights = weightss.to_numpy() 
weights = weights[:,0] 

In [ ]:
# sums for <2> using the sum way, equation 5 in https://arxiv.org/pdf/1010.0233

#version with for loops
def summm2(a, n):
    M=len(a)  
    if M<=1:
        return 0
    else:
        sumy=0 #set sum to zero
        for i in range(M-1):
            for jj in range(i+1, M):
                # sumy+=a[i]-a[j]
                sumy+=np.exp(1j *n*(a[i]-a[jj]))
        return sumy /comb(M, 2)
#Vectorized version of above which is much faster.  
def summm2v(a, n):
    #a = np.asarray(a)
    length = len(a)
    if length <= 1:
        return 0 
    # Create a 2D array of differences using broadcasting
    diff_matrix = a[:, None] - a[None, :]
    # Mask out the diagonal and lower triangle to avoid double-counting and self-pairs
    mask = np.triu(np.ones((length, length), dtype=bool), k=1)
    # Apply the exponential and sum only the upper triangle
    sumy = np.sum(np.exp(1j * n * diff_matrix[mask])) 
    return sumy /comb(length, 2)
summm2v(np.array([1,2,3,4]))


In [1]:
#same as above but using moments
#equation 4
def Qmoment(a, n): 
    return np.sum(np.exp(1j*n*a))

#equation 16
def cumulant_2(a, n): 
    M= len(a) 
    if M<=1:
        return 0
    return (np.abs(Qmoment(a, n))**2-M)/( (M-1)*M )

In [ ]:
# Slow way of four particle corrilation, equation 6
import numpy as np
import math
import itertool

def compute_four_particle_correlation(phi_list, n):
    """
    Compute ⟨4⟩ = (1 / P_{M,4}) * Σ' exp(i n (φ_i + φ_j - φ_k - φ_l))
    
    Parameters:
    phi_list : list or numpy array of phase angles (in radians)
    n        : integer harmonic number 
    
    Returns:
    complex number representing ⟨4⟩
    """
    M = len(phi_list)
    # Compute P_{M,4} = M! / (M-4)!
    P_M4 = math.factorial(M) / math.factorial(M - 4)
    
    # Generate all distinct combinations of 4 indices
    indices = range(M)
    total_sum = 0.0 + 0.0j
    
    for i, j, k, l in itertools.permutations(indices, 4):
        term = np.exp(1j * n * (phi_list[i] + phi_list[j] - phi_list[k] - phi_list[l]))
        # term = (phi_list[i] + phi_list[j] + phi_list[k] + phi_list[l])
        total_sum += term
    
    # Normalize by P_{M,4}
    return total_sum / P_M4 

#Perhaps faster way with vectors, nOT TESTED YETt
def compute_four_particle_correlation(phi_list, n):
    """
    Compute ⟨4⟩ = (1 / P_{M,4}) * Σ' exp(i n (φ_i + φ_j - φ_k - φ_l))
    
    Parameters:
    phi_list : list or numpy array of phase angles (in radians)
    n        : integer harmonic number 
    
    Returns:
    complex number representing ⟨4⟩
    """
    phi = np.asarray(phi_list)
    M = len(phi)
    
    # Compute normalization factor P_{M,4}
    P_M4 = math.factorial(M) / math.factorial(M - 4)
    
    # Generate all permutations of 4 indices
    perms = np.array(list(itertools.permutations(range(M), 4)))
    
    # Compute the sum using vectorized operations
    angles = phi[perms[:, 0]] + phi[perms[:, 1]] - phi[perms[:, 2]] - phi[perms[:, 3]]
    total_sum = np.sum(np.exp(1j * n * angles))
    return total_sum/P_M4

In [ ]:
#same as above, but with moments, equation 18 in the same paper. 

def order4(a, n):
    M = len(a)
    if M<=3:
        return 0
    Q2 = Qmoment(a, n)
    Q4 = Qmoment(a, 2*n)
    top = np.abs(Q2)**4+np.abs(Q4)**2-2*np.real(Q4* np.conjugate(Q2)*np.conjugate(Q2))
    bottom = 2*(M-2) *np.abs(Q2)**2-M*(M-3)
    normm = M*(M-1)*(M-2)*(M-3) 
    return (top-2*bottom)/normm

In [ ]:
#sample code to run itt.. 

cumulant2=np.zeros(10000, dtype =complex ) 
cum_moment = np.zeros(10000)
sum_weight = np.sum(weights )  
import time
start_time = time.time()
for i in range(1):
    store1 = np.array(vtrackphi_test[i][0])  
    cumulant2[i] = compute_four_particle_correlationv(store1, 2)
    cum_moment[i] = order_4(store1 )
#cumulant2*weights/sum_weight
# cum_moment *weights/sum_weight
 
end_time = time.time() 
print(cumulant2)
print(cum_moment)
print(cum_moment -cumulant2)
print(f"Total runtime: {end_time - start_time} seconds") 